In [ ]:
# !pip install emoji
# !pip install python-Levenshtein
# !pip install keras

## **Import Laibraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import style
import matplotlib.pyplot as plt
# from google.colab import drive
from keras.layers import Dense, Activation
import os
# from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from keras.models import Model
# import pickle
# from keras.layers import LSTM, Activation, Dense, Input, Embedding, SpatialDropout1D
# # from keras.layers import LSTM, Activation, Dense, Droupout, Input, Embedding, SpatialDropout1D
# from keras.optimizers import RMSprop
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing import sequence
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Sequential
# from keras.utils import pad_sequences
# import nltk
# import re
# import string
# from bs4 import BeautifulSoup
# import emoji
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# import gensim
# from gensim.utils import simple_preprocess
# import spacy
# from nltk.metrics import edit_distance
# from nltk.sentiment import SentimentIntensityAnalyzer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk import word_tokenize, pos_tag, ne_chunk
# from nltk.chunk import conlltags2tree, tree2conlltags
# from sklearn. feature_extraction.text import TfidfTransformer
# from sklearn.metrics import confusion_matrix
# from nltk.stem import WordNetLemmatizer
# from wordcloud import WordCloud
# from nltk.corpus import stopwords
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
nltk.download('vader_lexicon')
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
# Download NLTK resources if not already downloaded
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
stopword = set(stopwords.words('english'))

# **Add File_Path**

In [ ]:
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/Colab Notebooks/NLP_COURSE of Nitash_Sir/Projects/Hate Speech and Offensive Language'
file_name = "labeled_data.csv"
file_path = os.path.join(folder_path, file_name)


In [ ]:
data = pd.read_csv(file_path)
data.head()

In [ ]:
data.shape

In [ ]:
data.drop("Unnamed: 0", axis = 1, inplace = True)
data.head()

## **Preprocessing Step**

In [ ]:
data['hate_speech'].value_counts()

In [ ]:
data['tweet'][0]

In [ ]:
data.sample(10)

In [ ]:
data[data['class']==0]

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
# Check doublicate values
data.duplicated().sum()

In [ ]:
# Check counts of "class"
data['class'].value_counts()

## **Add new Features**

Adding a new column to this dataset as labels which will contain the values as:

Hate Speech                                                       
Offensive Language                                       
No Hate and Offensive

In [ ]:
data['labels'] = data['class'].map({
    0: "Hate Speech",
    1: "Offensive Language",
    2: "No Hate and No offensive"
})
data.drop("class", axis = 1, inplace= True)
data.head()

In [ ]:
df = data[["tweet", "labels"]]
df.head()

## **EDA and Vissualization**

In [ ]:
sns.countplot(x = 'hate_speech', data = data)

In [ ]:
sns.countplot(x = 'offensive_language', data = data)

In [ ]:
# to visualize the data using a count plot
# fig = plt.figure(figsize=(5,5))
ax = sns.countplot(x = 'labels', data = data)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# visualizing data using piechart
fig = plt.figure(figsize=(7,7))
colors = ('red', 'green', 'blue')
wp = {'linewidth':2, "edgecolor":'black'}
tags = df['labels'].value_counts()
explode=(0.1,0.1,0.1)
tags.plot(kind='pie', autopct="%1.1f%%", shadow=True, colors=colors, startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title("Distribution of sentiments")

In [ ]:
# visualizing non-hate tweets
non_hate_tweets = df[df.labels=='No Hate and No offensive']
non_hate_tweets.head()

In [ ]:
non_hate_tweets.value_counts()

In [ ]:
# visualizing non-hate tweets using word cloud
text=''.join([word for word in non_hate_tweets['tweet']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud=WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Most frequent words in non hate tweets", fontsize=19)
plt.show()

## **Feacture_Extractions**

Remove URLs & Punctuation and Handle emojies

In [ ]:
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Remove URLs
    pattern = re.compile(r"https?://\S+/www/\S+")
    text = pattern.sub('', text)

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Handle emojis
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]|[\U00002702-\U000027B0]", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)

    # Remove punctuation
    exclude = set(string.punctuation)
    text = text.translate(str.maketrans("", "", ''.join(exclude)))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered_words = [word for word in tokens if word.lower() not in stop_words]

    # Join the filtered words
    filtered_text = ' '.join(filtered_words)

    return filtered_text

# Apply preprocessing to "tweet" column
df["tweet"] = df["tweet"].apply(preprocess_text)


Tokenization and Stemming

In [ ]:
def tokenize_and_stem(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()

    # Remove stopwords and apply stemming
    stemmed_words = [stemmer.stem(word) for word in words if word.lower() not in stopwords.words('english')]

    # Join the stemmed words back into a single string
    stemmed_text = ' '.join(stemmed_words)

    return words

df["tweet"] = df["tweet"].apply(tokenize_and_stem)
# Join the list of tokens into a single string for each tweet
df["tweet"] = df["tweet"].apply(lambda tokens: " ".join(tokens))

In [ ]:
df["tweet"].shape

In [ ]:
df["tweet"]

## **TFIDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(texts):
    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

    # Fit and transform the input texts
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

    return tfidf_matrix

# Usage example:
tfidf_matrix = tfidf(df["tweet"])


In [ ]:
tfidf_matrix.shape

In [ ]:
X = tfidf_matrix
y = df['labels']


## **Train_Test_Split**

In [ ]:
# Step 2: Split your data into train and test sets
X = df["tweet"]
y = df['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Step 3: Vectorize your text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
print(tfidf_matrix.shape)

In [ ]:
# Step 4: Train your RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(tfidf_matrix, y_train)
y_pred = rf.predict(tfidf_vectorizer.transform(X_test))
rf_acc = accuracy_score(y_pred, y_test)

In [ ]:
rf_acc

In [ ]:
# Step 5: Save the tokenizer and RandomForestClassifier model
with open('tokenizer_stem.pickle', 'wb') as handle:
    pickle.dump(tokenize_and_stem, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tfidf_vectorizer.pickle', 'wb') as handle:
    pickle.dump(tfidf_vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('random_forest_model.pickle', 'wb') as handle:
    pickle.dump(rf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Step 6: Define and compile a simple Keras model (adjust as needed)
model = Sequential()
model.add(Dense(64, input_shape=(tfidf_matrix.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 7: Train the Keras model
model.fit(tfidf_matrix.toarray(), y_train, epochs=10)

In [ ]:
# Step 8: Save the Keras model
model.save("keras_model.h5")

In [ ]:
# Step 9: Load the models and tokenizer
loaded_tokenizer = pickle.load(open('tokenizer_stem.pickle', 'rb'))
loaded_tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pickle', 'rb'))
loaded_rf_model = pickle.load(open('random_forest_model.pickle', 'rb'))
loaded_keras_model = keras.models.load_model("keras_model.h5")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test

In [ ]:
print(X_train[0].shape)

## **MODEL_TRAINING AND TESTING**

## **1) LogisticRegression**

In [ ]:
# logreg = LogisticRegression()
# logreg.fit(X_train, y_train) #loading x_train and y_train data on model
# logreg_predict = logreg.predict(X_test) #predicting the value for test data

In [ ]:
# logreg_acc = accuracy_score(logreg_predict, y_test)
# print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
# #printing confusion matrix and classification report
# print(confusion_matrix(y_test, logreg_predict))
# print("\n")
# print(classification_report(y_test, logreg_predict))

In [ ]:
# #for visualizing confusion matrix
# style.use('classic')
# cm = confusion_matrix(y_test, logreg_predict, labels=logreg.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logreg.classes_)
# disp.plot()

## **2) GridSearchCV**

In [ ]:
#for performing hyperparameter tuning
# from sklearn.model_selection import GridSearchCV
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver' :['newton-cg', 'lbfgs','liblinear']}
# grid = GridSearchCV(LogisticRegression(), param_grid, cv = 5)
# grid.fit(X_train, y_train)
# print("Best Cross validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)

In [ ]:
# log_grid_pred = grid.predict(X_test)

In [ ]:
# log_grid_acc = accuracy_score(log_grid_pred, y_test)
# print("Test accuracy: {:.2f}%".format(log_grid_acc*100)) #printing model accuracy after applying hyperparamenter tuning

In [ ]:
# print(confusion_matrix(y_test, log_grid_pred))
# print("\n")
# print(classification_report(y_test, log_grid_pred))

In [ ]:
#for visualizing confusion matrix
# style.use('classic')
# cm = confusion_matrix(y_test, log_grid_pred, labels=grid.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid.classes_)
# disp.plot()

## **2. Decision Tree Classifier**

In [ ]:
# #for training the data on decision tree classifier model
# dtree = DecisionTreeClassifier()
# dtree.fit(X_train, y_train) #loading x_train and y_train data on model
# dtree_predict = dtree.predict(X_test) #predicting the value for test data
# dtree_acc = accuracy_score(dtree_predict, y_test)

In [ ]:
# print("Test accuracy: {:.2f}%".format(dtree_acc*100)) #printing accuracy of the model

In [ ]:
# #printing confusion matrix and classification report
# print(confusion_matrix(y_test, dtree_predict))
# print("\n")
# print(classification_report(y_test, dtree_predict))

In [ ]:
# #for visualizing confusion matrix
# style.use('classic')
# cm = confusion_matrix(y_test, dtree_predict, labels=dtree.classes)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtree.classes_)
# disp.plot()

**3. RandomForest Classifier**

In [ ]:
sddsff

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pre = rf.predict(X_test)
rf_acc = accuracy_score(y_pre, y_test)

In [ ]:
print("Test accuracy: {:.2f}%".format(rf_acc*100)) #printing accuracy of the model


In [ ]:
#printing confusion matrix and classification report
print(confusion_matrix(y_test, y_pre))
print("\n")
print(classification_report(y_test, y_pre))

In [ ]:
#for visualizing confusion matrix
style.use('classic')
cm = confusion_matrix(y_test, logreg_predict, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()

In [ ]:
X_train.shape, y_train.shape

## **Pickle DataSet**

In [ ]:
# Save the scikit-learn model
with open('rf_model.pickle', 'wb') as handle:
    pickle.dump(rf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the scikit-learn model
with open('rf_model.pickle', 'rb') as handle:
    loaded_rf = pickle.load(handle)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

# Define the input dimension to match the TF-IDF vector dimension
input_dim = tfidf_matrix.shape[1]

# Create a simple Keras model
model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the Keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(tfidf_matrix.toarray(), y_train, epochs=5)  # Use tfidf_matrix for training

# Save the Keras model
model.save('keras_model.h5')

# Load the Keras model
from keras.models import load_model
loaded_keras_model = load_model('keras_model.h5')


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

# Create a simple Keras model (example)
model = Sequential()
model.add(Dense(64, input_dim=100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the Keras model (example)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

# Save the Keras model
model.save('keras_model.h5')

# Load the Keras model
from keras.models import load_model
loaded_keras_model = load_model('keras_model.h5')


In [ ]:
import pickle
with open('tokenizer_stem.pickle', 'wb') as handle:
  pickle.dump(tokenize_and_stem, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('preprocess_text.pickle', 'wb') as handle:
  pickle.dump(preprocess_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
from keras.models import load_model
# Let's save the mdoel.
rf.save("model.h5")
import keras
load_model=keras.models.load_model("model.h5")
with open('tokenize_and_stem.pickle', 'rb') as handle:
  load_tokenizer = pickle.load(handle)